In [1]:
import os
import pandas as pd
import json
from IPython.display import display, Markdown
from tqdm.auto import tqdm

from common.consts import RESULTS_DIR, EVAL_SIZE
from common.utils import filename_to_obj, remove_index

/home/tdolega/miniconda/envs/p311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def results_as_pandas(filename):
    path = os.path.join(RESULTS_DIR, filename)
    with open(path, "r") as f:
        data = f.readlines()
    data = [json.loads(d) for d in data]
    data = pd.DataFrame(data)

    if len(data) == 0:
        print(f"empty file: {filename}")
        return data

    params = filename_to_obj(filename)
    for k, v in params.items():
        data[k] = v

    data = data.explode("evaluations")
    data = data.rename_axis("question_idx").reset_index()

    data = pd.concat([data, data["evaluations"].apply(pd.Series)], axis=1)
    evaluation_keys = data["evaluations"].apply(pd.Series).columns
    for col in evaluation_keys:
        data = pd.concat([data, data[col].apply(pd.Series).add_prefix(f"{col}/")], axis=1)
        data = data.drop(columns=col)
    data = data.drop(columns=["evaluations"])

    return data


files = os.listdir(RESULTS_DIR)
params_names = list(filename_to_obj(files[0]).keys())
all_results = pd.concat([results_as_pandas(f) for f in tqdm(files)])
all_results.head()

100%|██████████| 39/39 [01:26<00:00,  2.22s/it]


,question_idx,question_id,llm,prompt_id,temperature,nli,ellm,sim,citations/ais_recall,citations/ais_precision,...,citations/supported,citations/citations,citations/correct_citations,citations/out_of_range,correctness/answer_overlap,correctness/answer_entail,correctness/citations_recall,correctness/citations_precision,quality/answer_relevance,quality/new_question
0,0,5abab42e55429955dce3eed2,Mistral-7B-Instruct-v0.2,1,0.8,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,0.250000,1.000,...,"[0, 0, 0, 1]","[[], [], [3], [4]]","[[], [], [True], [True]]","[0, 0, 0, 0]",1.0,1.0,1.0,1.0,0.649022,Which of P.K. Subban's brothers was drafted fi...
1,0,5abab42e55429955dce3eed2,Mistral-7B-Instruct-v0.2,1,0.8,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,0.083333,0.875,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[], [], [3], [4], [6], [], [], [], [], [4, 5]...","[[], [], [True], [True], [True], [], [], [], [...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1.0,0.0,1.0,0.5,0.757470,"Who are the two professional hockey players, b..."
2,0,5abab42e55429955dce3eed2,Mistral-7B-Instruct-v0.2,1,0.8,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,0.000000,1.000,...,"[0, 0, 0, 0, 0, 0]","[[], [], [4], [6], [], []]","[[], [], [True], [True], [], []]","[0, 0, 0, 0, 0, 0]",0.5,0.0,0.5,0.5,0.770844,"Who are the two professional hockey players, b..."
3,1,5a761900554299109176e648,Mistral-7B-Instruct-v0.2,1,0.8,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,1.000000,1.000,...,[1],[[1]],[[True]],[0],1.0,1.0,0.5,1.0,0.450042,What is the name of the intelligence firm that...
4,1,5a761900554299109176e648,Mistral-7B-Instruct-v0.2,1,0.8,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,1.000000,1.000,...,"[1, 1]","[[0, 1], [0, 1]]","[[True, True], [True, True]]","[0, 0]",1.0,1.0,1.0,1.0,0.772082,What is the name of the lobbying group founded...


In [3]:
all_obj_cols = all_results.select_dtypes(include=["object"]).columns
drop_obj_cols = list(set(all_obj_cols) - set(params_names))
drop_obj_cols.remove("question_id")
print(f"Dropping columns: {drop_obj_cols}")
all_num_results = all_results.drop(columns=drop_obj_cols)

Dropping columns: ['citations/supported', 'citations/sentences', 'citations/out_of_range', 'quality/new_question', 'citations/citations', 'citations/correct_citations']


In [4]:
eval_split = all_num_results[all_num_results["question_idx"] < EVAL_SIZE]
train_split = all_num_results[all_num_results["question_idx"] >= EVAL_SIZE]

In [5]:
def aggregate(split):
    split = split.drop(columns=["question_idx"])
    results_with_std_for_each_question = split.groupby([*params_names, "question_id"]).agg(["mean", "std"])
    results_for_each_model = results_with_std_for_each_question.groupby(params_names)
    results = results_for_each_model.mean()
    results["n_questions"] = results_for_each_model.size()
    return results


eval_results = aggregate(eval_split)
train_results = aggregate(train_split)

if eval_results["n_questions"].nunique() != 1:
    print("Warning: not all rows in evaluation have the same number of examples")

In [6]:
display(Markdown("### Prompts comparison"))
parameter_results = eval_results[eval_results.index.get_level_values("llm") == "Mistral-7B-Instruct-v0.2"]


def show_cleaned_results(short_eval_display, keep_index_name=None):
    short_eval_display = short_eval_display.copy()
    for index_name in ["temperature", "nli", "ellm", "sim", "prompt_id"]:
        if index_name == keep_index_name:
            continue
        short_eval_display = remove_index(short_eval_display, index_name)
    important_columns = ["citations/ais_recall", "citations/ais_precision", "correctness/answer_overlap", "correctness/answer_entail", "correctness/citations_recall", "correctness/citations_precision", "quality/answer_relevance"]
    short_eval_display = short_eval_display[important_columns]
    return short_eval_display


show_cleaned_results(parameter_results[parameter_results.index.get_level_values("temperature") == "0.1"], keep_index_name="prompt_id")

### Prompts comparison

citations/ais_recall            \
                                                   mean       std   
llm                      prompt_id                                  
Mistral-7B-Instruct-v0.2 1                     0.575644  0.036560   
                         2                     0.501540  0.066146   
                         3                     0.580222  0.045033   

                                   citations/ais_precision            \
                                                      mean       std   
llm                      prompt_id                                     
Mistral-7B-Instruct-v0.2 1                        0.871517  0.038350   
                         2                        0.862167  0.034978   
                         3                        0.890076  0.024787   

                                   correctness/answer_overlap            \
                                                         mean       std   
llm                      prompt_id                                        
Mistral-7B-Instruct-v0.2 1                           0.872812  0.014915   
                         2                           0.855380  0.027905   
                         3                           0.855936  0.017321   

                                   correctness/answer_entail            \
                                                        mean       std   
llm                      prompt_id                                       
Mistral-7B-Instruct-v0.2 1                          0.886667  0.017321   
                         2                          0.860000  0.023094   
                         3                          0.863333  0.011547   

                                   correctness/citations_recall            \
                                                           mean       std   
llm                      prompt_id                                          
Mistral-7B-Instruct-v0.2 1                             0.751667  0.025981   
                         2                             0.770000  0.043301   
                         3                             0.665000  0.034641   

                                   correctness/citations_precision            \
                                                              mean       std   
llm                      prompt_id                                             
Mistral-7B-Instruct-v0.2 1                                0.721619  0.050125   
                         2                                0.670627  0.046655   
                         3                                0.658333  0.042370   

                                   quality/answer_relevance            
                                                       mean       std  
llm                      prompt_id                                     
Mistral-7B-Instruct-v0.2 1                         0.720431  0.028340  
                         2                         0.704898  0.030174  
                         3                         0.658884  0.027177

In [7]:
display(Markdown("### Temperature comparison"))
show_cleaned_results(parameter_results[parameter_results.index.get_level_values("prompt_id") == "1"], keep_index_name="temperature")

### Temperature comparison

citations/ais_recall            \
                                                     mean       std   
llm                      temperature                                  
Mistral-7B-Instruct-v0.2 0.01                    0.574825  0.027143   
                         0.1                     0.575644  0.036560   
                         0.2                     0.544493  0.081991   
                         0.3                     0.566712  0.084729   
                         0.4                     0.580572  0.150051   
                         0.5                     0.569651  0.144597   
                         0.6                     0.519373  0.168079   
                         0.7                     0.555853  0.182757   
                         0.8                     0.505895  0.195393   
                         0.9                     0.572745  0.204617   
                         1.0                     0.550218  0.202885   
                         1.5                     0.524532  0.221513   
                         2.0                     0.463269  0.259358   
                         2.5                     0.266854  0.262768   

                                     citations/ais_precision            \
                                                        mean       std   
llm                      temperature                                     
Mistral-7B-Instruct-v0.2 0.01                       0.889365  0.016922   
                         0.1                        0.871517  0.038350   
                         0.2                        0.877352  0.050804   
                         0.3                        0.863762  0.066155   
                         0.4                        0.877947  0.108934   
                         0.5                        0.881010  0.085049   
                         0.6                        0.856773  0.110804   
                         0.7                        0.858316  0.115458   
                         0.8                        0.842402  0.131938   
                         0.9                        0.876745  0.127270   
                         1.0                        0.858579  0.139181   
                         1.5                        0.856628  0.156983   
                         2.0                        0.785161  0.220605   
                         2.5                        0.712875  0.240513   

                                     correctness/answer_overlap            \
                                                           mean       std   
llm                      temperature                                        
Mistral-7B-Instruct-v0.2 0.01                          0.858923  0.012990   
                         0.1                           0.872812  0.014915   
                         0.2                           0.864735  0.029793   
                         0.3                           0.872534  0.045707   
                         0.4                           0.872336  0.044607   
                         0.5                           0.852962  0.073768   
                         0.6                           0.845013  0.049038   
                         0.7                           0.871201  0.047054   
                         0.8                           0.867148  0.062564   
                         0.9                           0.829410  0.074434   
                         1.0                           0.824547  0.066776   
                         1.5                           0.838289  0.074745   
                         2.0                           0.809466  0.089997   
                         2.5                           0.724706  0.186147   

                                     correctness/answer_entail            \
                                                          mean       std   
llm                      temperature                                       
Mistral-7B-Instruct-v0.2 0.01   

In [8]:
display(Markdown("### Evaluation results"))
eval_display = eval_results[eval_results.index.get_level_values("prompt_id") == "1"]
eval_display = eval_display[eval_display.index.get_level_values("temperature") == "0.1"]
eval_display = eval_display[eval_display.index.get_level_values("llm") != "Mixtral-8x7B-Instruct-v0.1.Q8_0"]  # use this model only for training data, because we already have same model without quantization
eval_display = remove_index(eval_display, "prompt_id")
eval_display = eval_display.sort_values(by=("correctness/citations_recall", "mean"), ascending=False)
eval_display

### Evaluation results

prompt_id  \
                                                                                                                        
llm                           temperature nli                     ellm                     sim                          
rag-tge_Llama-3-8B_v1         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
rag-tge_Mistral_v2-4480       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
rag-tge_Mistral_v2-3360       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
rag-tge_Mistral.Q8            0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gpt-3.5-turbo-0125            0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mixtral-8x7B-Instruct-v0.1    0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gpt-4-turbo                   0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Meta-Llama-3-70B-Instruct     0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
zephyr-orpo-141b-A35b-v0.1    0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Meta-Llama-3-8B-Instruct      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mistral-7B-Instruct-v0.2      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-110b-chat             0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-14b-chat-q8_0         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-32b-chat-q8_0         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
zephyr-7b-beta                0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
c4ai-command-r-plus           0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-7b-chat-q8_0          0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
rag-tge_TinyLlama.Q32         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gemma-1.1-7b-it               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Phi-3-mini-4k-instruct        0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
tinyllama-1.1b-chat-v1.0.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gemma-1.1-2b-it               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mistral-7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   

                                                                                                            citations/ais_recall  \
                                                                                                                            mean   
llm                           temperature nli                     ellm                     sim                                     
rag-tge_Llama-3-8B_v1         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.640377   
rag-tge_Mistral_v2-4480       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.733170   
rag-tge_Mistral_v2-3360       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.682667   
rag-tge_Mistral.Q8            0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.812135   
gpt-3.5-turbo-0125            0.1    

In [9]:
clean_results = show_cleaned_results(eval_display)
for sort_by in [
    ("citations/ais_recall", "mean"),
    ("citations/ais_precision", "mean"),
    ("correctness/answer_overlap", "mean"),
    ("correctness/answer_entail", "mean"),
    ("correctness/citations_recall", "mean"),
    ("correctness/citations_precision", "mean"),
    ("quality/answer_relevance", "mean"),
]:
    display(Markdown(f"### sorted by: {sort_by[0]}"))
    display(clean_results.sort_values(by=sort_by, ascending=False))

### sorted by: citations/ais_recall

citations/ais_recall            \
                                              mean       std   
llm                                                            
rag-tge_Mistral.Q8                        0.812135  0.060520   
gpt-3.5-turbo-0125                        0.737556  0.072604   
rag-tge_Mistral_v2-4480                   0.733170  0.134646   
gpt-4-turbo                               0.733013  0.045401   
Meta-Llama-3-70B-Instruct                 0.691944  0.049556   
rag-tge_Mistral_v2-3360                   0.682667  0.146211   
Mixtral-8x7B-Instruct-v0.1                0.653500  0.061199   
rag-tge_Llama-3-8B_v1                     0.640377  0.100349   
qwen1_5-14b-chat-q8_0                     0.633333  0.042339   
Mistral-7B-Instruct-v0.2                  0.575644  0.036560   
qwen1_5-110b-chat                         0.528167  0.095486   
qwen1_5-32b-chat-q8_0                     0.512222  0.031856   
Meta-Llama-3-8B-Instruct                  0.492702  0.060547   
zephyr-orpo-141b-A35b-v0.1                0.417516  0.134222   
c4ai-command-r-plus                       0.409167  0.060382   
zephyr-7b-beta                            0.406690  0.090558   
rag-tge_TinyLlama.Q32                     0.405833  0.121770   
gemma-1.1-7b-it                           0.367889  0.122999   
Phi-3-mini-4k-instruct                    0.252796  0.030503   
qwen1_5-7b-chat-q8_0                      0.233444  0.063671   
tinyllama-1.1b-chat-v1.0.Q8_0             0.041361  0.010585   
gemma-1.1-2b-it                           0.006667  0.005774   
Mistral-7B-Instruct-v0.1                  0.005000  0.000000   

                              citations/ais_precision            \
                                                 mean       std   
llm                                                               
rag-tge_Mistral.Q8                           0.957667  0.021027   
gpt-3.5-turbo-0125                           0.995833  0.002887   
rag-tge_Mistral_v2-4480                      0.954283  0.031338   
gpt-4-turbo                                  0.985000  0.005774   
Meta-Llama-3-70B-Instruct                    0.955556  0.017802   
rag-tge_Mistral_v2-3360                      0.957922  0.032981   
Mixtral-8x7B-Instruct-v0.1                   0.928603  0.029692   
rag-tge_Llama-3-8B_v1                        0.953748  0.030020   
qwen1_5-14b-chat-q8_0                        0.975000  0.008660   
Mistral-7B-Instruct-v0.2                     0.871517  0.038350   
qwen1_5-110b-chat                            0.976111  0.026943   
qwen1_5-32b-chat-q8_0                        0.993333  0.002887   
Meta-Llama-3-8B-Instruct                     0.951210  0.022033   
zephyr-orpo-141b-A35b-v0.1                   0.796321  0.101052   
c4ai-command-r-plus                          0.714694  0.061399   
zephyr-7b-beta                               0.910524  0.068665   
rag-tge_TinyLlama.Q32                        0.749778  0.039015   
gemma-1.1-7b-it                              0.766667  0.082995   
Phi-3-mini-4k-instruct                       0.578455  0.069580   
qwen1_5-7b-chat-q8_0                         0.862000  0.057158   
tinyllama-1.1b-chat-v1.0.Q8_0                0.125000  0.031754   
gemma-1.1-2b-it                              0.046111  0.009623   
Mistral-7B-Instruct-v0.1                     0.010000  0.000000   

                              correctness/answer_overlap            \
                                                    mean       std   
llm                                                                  
rag-tge_Mistral.Q8                              0.860838  0.012892   
gpt-3.5-turbo-0125                              0.884244  0.012990   
rag-tge_Mistral_v2-4480                         0.915308  0.017918   
gpt-4-turbo                                     0.877302  0.007661   
Meta-Llama-3-70B-Instruct                       0.865923  0.020651   
rag-tge_Mistral_v2-3360                         0.936128  0.03

### sorted by: citations/ais_precision

citations/ais_recall            \
                                              mean       std   
llm                                                            
gpt-3.5-turbo-0125                        0.737556  0.072604   
qwen1_5-32b-chat-q8_0                     0.512222  0.031856   
gpt-4-turbo                               0.733013  0.045401   
qwen1_5-110b-chat                         0.528167  0.095486   
qwen1_5-14b-chat-q8_0                     0.633333  0.042339   
rag-tge_Mistral_v2-3360                   0.682667  0.146211   
rag-tge_Mistral.Q8                        0.812135  0.060520   
Meta-Llama-3-70B-Instruct                 0.691944  0.049556   
rag-tge_Mistral_v2-4480                   0.733170  0.134646   
rag-tge_Llama-3-8B_v1                     0.640377  0.100349   
Meta-Llama-3-8B-Instruct                  0.492702  0.060547   
Mixtral-8x7B-Instruct-v0.1                0.653500  0.061199   
zephyr-7b-beta                            0.406690  0.090558   
Mistral-7B-Instruct-v0.2                  0.575644  0.036560   
qwen1_5-7b-chat-q8_0                      0.233444  0.063671   
zephyr-orpo-141b-A35b-v0.1                0.417516  0.134222   
gemma-1.1-7b-it                           0.367889  0.122999   
rag-tge_TinyLlama.Q32                     0.405833  0.121770   
c4ai-command-r-plus                       0.409167  0.060382   
Phi-3-mini-4k-instruct                    0.252796  0.030503   
tinyllama-1.1b-chat-v1.0.Q8_0             0.041361  0.010585   
gemma-1.1-2b-it                           0.006667  0.005774   
Mistral-7B-Instruct-v0.1                  0.005000  0.000000   

                              citations/ais_precision            \
                                                 mean       std   
llm                                                               
gpt-3.5-turbo-0125                           0.995833  0.002887   
qwen1_5-32b-chat-q8_0                        0.993333  0.002887   
gpt-4-turbo                                  0.985000  0.005774   
qwen1_5-110b-chat                            0.976111  0.026943   
qwen1_5-14b-chat-q8_0                        0.975000  0.008660   
rag-tge_Mistral_v2-3360                      0.957922  0.032981   
rag-tge_Mistral.Q8                           0.957667  0.021027   
Meta-Llama-3-70B-Instruct                    0.955556  0.017802   
rag-tge_Mistral_v2-4480                      0.954283  0.031338   
rag-tge_Llama-3-8B_v1                        0.953748  0.030020   
Meta-Llama-3-8B-Instruct                     0.951210  0.022033   
Mixtral-8x7B-Instruct-v0.1                   0.928603  0.029692   
zephyr-7b-beta                               0.910524  0.068665   
Mistral-7B-Instruct-v0.2                     0.871517  0.038350   
qwen1_5-7b-chat-q8_0                         0.862000  0.057158   
zephyr-orpo-141b-A35b-v0.1                   0.796321  0.101052   
gemma-1.1-7b-it                              0.766667  0.082995   
rag-tge_TinyLlama.Q32                        0.749778  0.039015   
c4ai-command-r-plus                          0.714694  0.061399   
Phi-3-mini-4k-instruct                       0.578455  0.069580   
tinyllama-1.1b-chat-v1.0.Q8_0                0.125000  0.031754   
gemma-1.1-2b-it                              0.046111  0.009623   
Mistral-7B-Instruct-v0.1                     0.010000  0.000000   

                              correctness/answer_overlap            \
                                                    mean       std   
llm                                                                  
gpt-3.5-turbo-0125                              0.884244  0.012990   
qwen1_5-32b-chat-q8_0                           0.873725  0.005774   
gpt-4-turbo                                     0.877302  0.007661   
qwen1_5-110b-chat                               0.867315  0.020207   
qwen1_5-14b-chat-q8_0                           0.876344  0.006598   
rag-tge_Mistral_v2-3360                         0.936128  0.03

### sorted by: correctness/answer_overlap

...

citations/ais_recall            \
                                              mean       std   
llm                                                            
rag-tge_Llama-3-8B_v1                     0.640377  0.100349   
rag-tge_Mistral_v2-4480                   0.733170  0.134646   
rag-tge_Mistral_v2-3360                   0.682667  0.146211   
rag-tge_Mistral.Q8                        0.812135  0.060520   
gpt-3.5-turbo-0125                        0.737556  0.072604   
Mixtral-8x7B-Instruct-v0.1                0.653500  0.061199   
gpt-4-turbo                               0.733013  0.045401   
Meta-Llama-3-70B-Instruct                 0.691944  0.049556   
zephyr-orpo-141b-A35b-v0.1                0.417516  0.134222   
Meta-Llama-3-8B-Instruct                  0.492702  0.060547   
Mistral-7B-Instruct-v0.2                  0.575644  0.036560   
qwen1_5-110b-chat                         0.528167  0.095486   
qwen1_5-14b-chat-q8_0                     0.633333  0.042339   
qwen1_5-32b-chat-q8_0                     0.512222  0.031856   
zephyr-7b-beta                            0.406690  0.090558   
c4ai-command-r-plus                       0.409167  0.060382   
qwen1_5-7b-chat-q8_0                      0.233444  0.063671   
rag-tge_TinyLlama.Q32                     0.405833  0.121770   
gemma-1.1-7b-it                           0.367889  0.122999   
Phi-3-mini-4k-instruct                    0.252796  0.030503   
tinyllama-1.1b-chat-v1.0.Q8_0             0.041361  0.010585   
gemma-1.1-2b-it                           0.006667  0.005774   
Mistral-7B-Instruct-v0.1                  0.005000  0.000000   

                              citations/ais_precision            \
                                                 mean       std   
llm                                                               
rag-tge_Llama-3-8B_v1                        0.953748  0.030020   
rag-tge_Mistral_v2-4480                      0.954283  0.031338   
rag-tge_Mistral_v2-3360                      0.957922  0.032981   
rag-tge_Mistral.Q8                           0.957667  0.021027   
gpt-3.5-turbo-0125                           0.995833  0.002887   
Mixtral-8x7B-Instruct-v0.1                   0.928603  0.029692   
gpt-4-turbo                                  0.985000  0.005774   
Meta-Llama-3-70B-Instruct                    0.955556  0.017802   
zephyr-orpo-141b-A35b-v0.1                   0.796321  0.101052   
Meta-Llama-3-8B-Instruct                     0.951210  0.022033   
Mistral-7B-Instruct-v0.2                     0.871517  0.038350   
qwen1_5-110b-chat                            0.976111  0.026943   
qwen1_5-14b-chat-q8_0                        0.975000  0.008660   
qwen1_5-32b-chat-q8_0                        0.993333  0.002887   
zephyr-7b-beta                               0.910524  0.068665   
c4ai-command-r-plus                          0.714694  0.061399   
qwen1_5-7b-chat-q8_0                         0.862000  0.057158   
rag-tge_TinyLlama.Q32                        0.749778  0.039015   
gemma-1.1-7b-it                              0.766667  0.082995   
Phi-3-mini-4k-instruct                       0.578455  0.069580   
tinyllama-1.1b-chat-v1.0.Q8_0                0.125000  0.031754   
gemma-1.1-2b-it                              0.046111  0.009623   
Mistral-7B-Instruct-v0.1                     0.010000  0.000000   

                              correctness/answer_overlap            \
                                                    mean       std   
llm                                                                  
rag-tge_Llama-3-8B_v1                           0.944436  0.013723   
rag-tge_Mistral_v2-4480                         0.915308  0.017918   
rag-tge_Mistral_v2-3360                         0.936128  0.030777   
rag-tge_Mistral.Q8                              0.860838  0.012892   
gpt-3.5-turbo-0125                              0.884244  0.012990   
Mixtral-8x7B-Instruct-v0.1                      0.909923  0.00

### sorted by: correctness/citations_precision

citations/ais_recall            \
                                              mean       std   
llm                                                            
qwen1_5-32b-chat-q8_0                     0.512222  0.031856   
gpt-3.5-turbo-0125                        0.737556  0.072604   
gpt-4-turbo                               0.733013  0.045401   
qwen1_5-110b-chat                         0.528167  0.095486   
qwen1_5-14b-chat-q8_0                     0.633333  0.042339   
Meta-Llama-3-70B-Instruct                 0.691944  0.049556   
Meta-Llama-3-8B-Instruct                  0.492702  0.060547   
rag-tge_Mistral.Q8                        0.812135  0.060520   
rag-tge_Mistral_v2-4480                   0.733170  0.134646   
Mixtral-8x7B-Instruct-v0.1                0.653500  0.061199   
rag-tge_Mistral_v2-3360                   0.682667  0.146211   
qwen1_5-7b-chat-q8_0                      0.233444  0.063671   
rag-tge_Llama-3-8B_v1                     0.640377  0.100349   
zephyr-7b-beta                            0.406690  0.090558   
Mistral-7B-Instruct-v0.2                  0.575644  0.036560   
gemma-1.1-7b-it                           0.367889  0.122999   
c4ai-command-r-plus                       0.409167  0.060382   
rag-tge_TinyLlama.Q32                     0.405833  0.121770   
zephyr-orpo-141b-A35b-v0.1                0.417516  0.134222   
Phi-3-mini-4k-instruct                    0.252796  0.030503   
gemma-1.1-2b-it                           0.006667  0.005774   
tinyllama-1.1b-chat-v1.0.Q8_0             0.041361  0.010585   
Mistral-7B-Instruct-v0.1                  0.005000  0.000000   

                              citations/ais_precision            \
                                                 mean       std   
llm                                                               
qwen1_5-32b-chat-q8_0                        0.993333  0.002887   
gpt-3.5-turbo-0125                           0.995833  0.002887   
gpt-4-turbo                                  0.985000  0.005774   
qwen1_5-110b-chat                            0.976111  0.026943   
qwen1_5-14b-chat-q8_0                        0.975000  0.008660   
Meta-Llama-3-70B-Instruct                    0.955556  0.017802   
Meta-Llama-3-8B-Instruct                     0.951210  0.022033   
rag-tge_Mistral.Q8                           0.957667  0.021027   
rag-tge_Mistral_v2-4480                      0.954283  0.031338   
Mixtral-8x7B-Instruct-v0.1                   0.928603  0.029692   
rag-tge_Mistral_v2-3360                      0.957922  0.032981   
qwen1_5-7b-chat-q8_0                         0.862000  0.057158   
rag-tge_Llama-3-8B_v1                        0.953748  0.030020   
zephyr-7b-beta                               0.910524  0.068665   
Mistral-7B-Instruct-v0.2                     0.871517  0.038350   
gemma-1.1-7b-it                              0.766667  0.082995   
c4ai-command-r-plus                          0.714694  0.061399   
rag-tge_TinyLlama.Q32                        0.749778  0.039015   
zephyr-orpo-141b-A35b-v0.1                   0.796321  0.101052   
Phi-3-mini-4k-instruct                       0.578455  0.069580   
gemma-1.1-2b-it                              0.046111  0.009623   
tinyllama-1.1b-chat-v1.0.Q8_0                0.125000  0.031754   
Mistral-7B-Instruct-v0.1                     0.010000  0.000000   

                              correctness/answer_overlap            \
                                                    mean       std   
llm                                                                  
qwen1_5-32b-chat-q8_0                           0.873725  0.005774   
gpt-3.5-turbo-0125                              0.884244  0.012990   
gpt-4-turbo                                     0.877302  0.007661   
qwen1_5-110b-chat                               0.867315  0.020207   
qwen1_5-14b-chat-q8_0                           0.876344  0.006598   
Meta-Llama-3-70B-Instruct                       0.865923  0.02

### sorted by: quality/answer_relevance

citations/ais_recall            \
                                              mean       std   
llm                                                            
qwen1_5-7b-chat-q8_0                      0.233444  0.063671   
Meta-Llama-3-70B-Instruct                 0.691944  0.049556   
gpt-3.5-turbo-0125                        0.737556  0.072604   
gpt-4-turbo                               0.733013  0.045401   
rag-tge_Mistral.Q8                        0.812135  0.060520   
gemma-1.1-7b-it                           0.367889  0.122999   
Mistral-7B-Instruct-v0.1                  0.005000  0.000000   
qwen1_5-110b-chat                         0.528167  0.095486   
rag-tge_Llama-3-8B_v1                     0.640377  0.100349   
rag-tge_Mistral_v2-4480                   0.733170  0.134646   
Mixtral-8x7B-Instruct-v0.1                0.653500  0.061199   
rag-tge_Mistral_v2-3360                   0.682667  0.146211   
Meta-Llama-3-8B-Instruct                  0.492702  0.060547   
Mistral-7B-Instruct-v0.2                  0.575644  0.036560   
qwen1_5-14b-chat-q8_0                     0.633333  0.042339   
zephyr-7b-beta                            0.406690  0.090558   
qwen1_5-32b-chat-q8_0                     0.512222  0.031856   
gemma-1.1-2b-it                           0.006667  0.005774   
zephyr-orpo-141b-A35b-v0.1                0.417516  0.134222   
Phi-3-mini-4k-instruct                    0.252796  0.030503   
c4ai-command-r-plus                       0.409167  0.060382   
rag-tge_TinyLlama.Q32                     0.405833  0.121770   
tinyllama-1.1b-chat-v1.0.Q8_0             0.041361  0.010585   

                              citations/ais_precision            \
                                                 mean       std   
llm                                                               
qwen1_5-7b-chat-q8_0                         0.862000  0.057158   
Meta-Llama-3-70B-Instruct                    0.955556  0.017802   
gpt-3.5-turbo-0125                           0.995833  0.002887   
gpt-4-turbo                                  0.985000  0.005774   
rag-tge_Mistral.Q8                           0.957667  0.021027   
gemma-1.1-7b-it                              0.766667  0.082995   
Mistral-7B-Instruct-v0.1                     0.010000  0.000000   
qwen1_5-110b-chat                            0.976111  0.026943   
rag-tge_Llama-3-8B_v1                        0.953748  0.030020   
rag-tge_Mistral_v2-4480                      0.954283  0.031338   
Mixtral-8x7B-Instruct-v0.1                   0.928603  0.029692   
rag-tge_Mistral_v2-3360                      0.957922  0.032981   
Meta-Llama-3-8B-Instruct                     0.951210  0.022033   
Mistral-7B-Instruct-v0.2                     0.871517  0.038350   
qwen1_5-14b-chat-q8_0                        0.975000  0.008660   
zephyr-7b-beta                               0.910524  0.068665   
qwen1_5-32b-chat-q8_0                        0.993333  0.002887   
gemma-1.1-2b-it                              0.046111  0.009623   
zephyr-orpo-141b-A35b-v0.1                   0.796321  0.101052   
Phi-3-mini-4k-instruct                       0.578455  0.069580   
c4ai-command-r-plus                          0.714694  0.061399   
rag-tge_TinyLlama.Q32                        0.749778  0.039015   
tinyllama-1.1b-chat-v1.0.Q8_0                0.125000  0.031754   

                              correctness/answer_overlap            \
                                                    mean       std   
llm                                                                  
qwen1_5-7b-chat-q8_0                            0.864821  0.024118   
Meta-Llama-3-70B-Instruct                       0.865923  0.020651   
gpt-3.5-turbo-0125                              0.884244  0.012990   
gpt-4-turbo                                     0.877302  0.007661   
rag-tge_Mistral.Q8                              0.860838  0.012892   
gemma-1.1-7b-it                                 0.722513  0.07

In [10]:
display(Markdown("### Training results"))
train_display = remove_index(train_results, "prompt_id")
train_display

### Training results

prompt_id  \
                                                                                                                          
llm                             temperature nli                     ellm                     sim                          
Meta-Llama-3-70B-Instruct       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mixtral-8x7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mixtral-8x7B-Instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-32b-chat-q8_0           0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
zephyr-orpo-141b-A35b-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   

                                                                                                              citations/ais_recall  \
                                                                                                                              mean   
llm                             temperature nli                     ellm                     sim                                     
Meta-Llama-3-70B-Instruct       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.742273   
Mixtral-8x7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.670310   
Mixtral-8x7B-Instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.623737   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.564920   
qwen1_5-32b-chat-q8_0           0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.549691   
zephyr-orpo-141b-A35b-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.443428   

                                                                                                                         \
                                                                                                                    std   
llm                             temperature nli                     ellm                     sim                          
Meta-Llama-3-70B-Instruct       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.031242   
Mixtral-8x7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.063516   
Mixtral-8x7B-Instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.070232   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.117130   
qwen1_5-32b-chat-q8_0           0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.031801   
zephyr-orpo-141b-A35b-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.127016   

                                                                                                              citations/ais_precision  \
                                                                                                                                 mean   
llm                             temperature nli                     ellm                     sim                                        
Meta-Llama-3-70B-Instruct       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2                0.958344   
Mixtral-8x7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2                0.931789   
Mi